# 03 全流程构建与解码演示
本 Notebook 将整合此前分散的脚本，完成：
1. 载入与聚合字典（`usrs/`：Chara.gb / TONEPY.txt / Pth.gb / HSK词性表）
2. 生成聚合字典文件 `resources/lexicon_aggregate.json`（含 base_pinyin_to_chars）
3. 基于 PFR 人民日报语料（示例：1998-01）统计字级 unigram / bigram + 发射计数
4. 根据统计结果构建 HMM 参数并保存 `resources/hmm_params.json`
5. 使用简短拼音序列做一次解码演示
6.（可选）校验：若关键文件存在则跳过重复构建（可手动强制重建）

注意：
- 发射计数当前策略为：字出现次数分配到其所有候选 base 拼音（不是真实多音字概率，需后续改进）。
- Pth.gb 已转为 UTF-8，本 Notebook 直接按 UTF-8 读取。
- 若语料较大，统计可能耗时；本示例仅用 199801。
- 可扩展新增词级 n-gram / Top-K Beam 等。

In [1]:
from pathlib import Path
import json, math, sys
from collections import Counter
sys.path.append('..')  # 允许导入 src.* 模块

from src.preprocess.load_lexicons import load_all, tone_to_base
from src.preprocess.build_stats import build_stats, attach_pinyin_emission
from src.models.hmm import HMMParams
from src.decoder.viterbi import viterbi_decode

BASE_DIR = Path('..').resolve()
USRS_DIR = BASE_DIR / 'usrs'
RES_DIR = BASE_DIR / 'resources'
CORPUS_FILE = USRS_DIR / 'peopledaily' / 'PeopleDaily199801.txt'
RES_DIR.mkdir(exist_ok=True)
print('Base:', BASE_DIR)
print('Corpus exists:', CORPUS_FILE.exists())

Base: E:\zz_save\大三上\NLP\Coursework 1
Corpus exists: True


## 1. 载入与聚合字典

In [2]:
lexicon_path = RES_DIR / 'lexicon_aggregate.json'
force_rebuild = False  # 如需强制重新生成设为 True
if force_rebuild or not lexicon_path.exists():
    lex_data = load_all(USRS_DIR)
    lexicon_path.write_text(json.dumps(lex_data, ensure_ascii=False, indent=2), encoding='utf-8')
    print('聚合字典已生成:', lexicon_path)
else:
    lex_data = json.loads(lexicon_path.read_text(encoding='utf-8'))
    print('聚合字典已存在，已加载。')
list(lex_data.keys())

聚合字典已存在，已加载。


['char_to_pinyins',
 'pinyin_tone_to_chars',
 'base_pinyin_to_chars',
 'word_to_pinyin_seq',
 'hsk_word_pos']

In [3]:
# 预览 base_pinyin_to_chars 中几个拼音的候选规模
base_map = lex_data['base_pinyin_to_chars']
for py in ['a','ai','shi','zhong','ren']:
    cand = base_map.get(py, [])
    print(py, '候选数:', len(cand), '示例:', ''.join(cand[:12]))

a 候选数: 7 示例: 阿啊呵腌吖锕嗄
ai 候选数: 23 示例: 哀挨埃唉哎捱锿呆癌皑矮蔼
shi 候选数: 68 示例: 师诗失施尸湿狮嘘虱蓍酾鲺
zhong 候选数: 17 示例: 中终钟忠衷锺盅忪螽舯种肿
ren 候选数: 17 示例: 人任仁壬忍稔荏认韧刃纫饪


## 2. 统计 unigram / bigram 以及发射计数
这里直接复用 `build_stats.py` 中的核心函数以避免再次写文件再读。
（若需命令行方式，可单独运行模块：见 `resources/README.md` 示例。）

In [4]:
# 统计字级频次
unigram, bigram = build_stats(CORPUS_FILE)
print('unigram 字种数量:', len(unigram), '总频次:', sum(unigram.values()))
print('bigram 数量:', len(bigram))
# 构造发射计数
emit_counts = attach_pinyin_emission(unigram, base_map)
print('发射计数字数:', len(emit_counts))
# 保存到文件，保持与脚本接口一致
def save_counter(counter, path: Path):
    obj = {'__type__':'counter','data':{(k if not isinstance(k, tuple) else '|'.join(k)):v for k,v in counter.items()}}
    path.write_text(json.dumps(obj, ensure_ascii=False), encoding='utf-8')
save_counter(unigram, RES_DIR / 'freq_unigram.json')
save_counter(bigram, RES_DIR / 'freq_bigram.json')
(RES_DIR / 'freq_emit.json').write_text(json.dumps({ch: dict(c.items()) for ch,c in emit_counts.items()}, ensure_ascii=False), encoding='utf-8')
print('统计持久化完成。')

unigram 字种数量: 4577 总频次: 1606385
bigram 数量: 314074
发射计数字数: 4557
统计持久化完成。


In [5]:
# 预览若干高频字
print('Top 15 unigram:')
for ch, cnt in unigram.most_common(15):
    print(ch, cnt)
# 预览某个字的发射候选（截断）
sample_char = unigram.most_common(1)[0][0]
emit_row = emit_counts.get(sample_char, {})
print('示例字:', sample_char, '发射拼音数:', len(emit_row))
list(list(emit_row.items())[:10])

Top 15 unigram:
的 55212
国 17901
一 17642
在 13701
中 12962
人 12586
了 12434
和 11945
是 11625
有 11193
年 11072
大 10966
不 9291
为 8886
会 8510
示例字: 的 发射拼音数: 2


[('de', 55212), ('di', 55212)]

## 3. 构建 HMM 参数
`HMMParams.from_frequency(unigram, bigram, emit)` 会：
- init: unigram 归一化
- trans: (a,b) / a + add-k 平滑
- emit: (char,pinyin)/sum(char,*) + add-k 平滑
注意：调用时顺序：unigram_path, bigram_path, emit_path（不要传错）。

In [6]:
hmm_params_path = RES_DIR / 'hmm_params.json'
hmm = HMMParams.from_frequency(RES_DIR / 'freq_unigram.json',
                               RES_DIR / 'freq_bigram.json',
                               RES_DIR / 'freq_emit.json')
hmm.save(hmm_params_path)
print('HMM 参数已保存:', hmm_params_path)
# 预览部分 init / trans / emit 尺寸
print('init 大小:', len(hmm.init_log_probs))
print('trans 起始状态数:', len(hmm.trans_log_probs))
print('emit 字数:', len(hmm.emit_log_probs))

HMM 参数已保存: E:\zz_save\大三上\NLP\Coursework 1_拼音（全拼）联想输入法\resources\hmm_params.json
init 大小: 4577
trans 起始状态数: 4577
emit 字数: 4557


## 4. 简单解码演示
选取一串常见拼音（需在 base_pinyin_to_chars 中）。实际输出质量依赖语料规模与映射准确度。

In [7]:
def decode_sequence(py_str: str):
    seq = py_str.split()
    result = viterbi_decode(seq, base_map, hmm)
    return seq, result
for test in ['zhong guo ren min', 'ren min ri bao', 'jing ji fa zhan', 'xin wen bao gao']:
    seq, out = decode_sequence(test)
    print('>',' '.join(seq),'=>', out)

> zhong guo ren min => 中国人民
> ren min ri bao => 人民日报
> jing ji fa zhan => 经济发展
> xin wen bao gao => 新闻报告


## 5. （可选）一致性快速校验
确保 HMM 反序列化后结果一致。

In [8]:
reloaded = HMMParams.load(hmm_params_path)
test_seq = ['zhong','guo','ren']
out1 = viterbi_decode(test_seq, base_map, hmm)
out2 = viterbi_decode(test_seq, base_map, reloaded)
assert out1 == out2, 'Reloaded HMM mismatch'
print('Reload check passed:', out1)

Reload check passed: 中国人


## 6. 后续改进建议
- 发射概率：利用 `word_to_pinyin_seq` 统计 (char,pinyin) 真实频率，而非复制式分配。
- 引入 Top-K / Beam：限制每层扩展宽度提升速度。
- 多音字截断：按 unigram 频次 / PMI 排序截取前 N。
- 词级语言模型：对 解码结果 -> 分词 -> 语言模型重排序。
- 评估：构建 (拼音输入, 参考汉字) 对，计算字符/词准确率。